In [2]:
!pip install pymupdf --quiet 

In [3]:
import boto3
import pprint
from botocore.client import Config
import json
import re

import os
import json
import time
import base64
from PIL import Image
import io
from IPython.display import Image
from IPython.display import display
import ipywidgets as widgets
#from skimage import io



pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_runtime = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-west-2


In [4]:
from helper.llmcall import * 
model = "anthropic.claude-3-sonnet"


# functionas

In [5]:
# Function to convert image to base64
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        binary_data = image_file.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode('utf-8')
    return base64_string

# PDF to Image

In [8]:
import fitz  # PyMuPDF, imported as fitz for backward compatibility reasons

file_path = "Image/pge-utility-bill.pdf"
doc = fitz.open(file_path)  # open document
for i, page in enumerate(doc):
    pix = page.get_pixmap()  # render page to an image
    pageindex = f"_{i+1}.png"
    #print(pageindex)
    imagepath = file_path.replace(".pdf",pageindex)
    pix.save(imagepath)


# using GenAI to read image

## sample

In [ ]:
# # Example usage
# image_path = 'Image/pge-utility-bill.pdf'  # Specify the path to your image
# base64_string = image_to_base64(image_path)
# Image(filename=image_path) 
# response_text = interactWithLLM("can you describe this image in details?",base64_string,llm_type)
# print('response_text --- \n' + response_text)

## read from folder

In [10]:
directory_path ='Image/electricity'  # Specify the path to your image

for path, folders, files  in os.walk(directory_path):
     for filename in files:
            image_path = os.path.join(directory_path, filename)
            print(image_path)
            base64_string = image_to_base64(image_path)
            Image(filename=image_path) 
            response_text = interactWithLLM(bedrock_runtime,prompt="how many kWh has been used for current period? only mentioned digits. if you do not have it return 0",image=base64_string,llm_type=model)
            print('response_text --- \n' + response_text)


    

Image/electricity/e-YHU utility bill.png
response_text --- 
The total electricity usage in kWh for the current billing period, as shown on the utility bill, is 1,353 kWh.
Image/electricity/e-24sample-bill-for-website.jpg
response_text --- 
The current period usage mentioned in digits is 573 kWh, according to the "Total Usage: 573 kWh from 12/12 to 01/12" line in the electricity bill.
Image/electricity/e-Sample-electric-Bill-2023.jpg
response_text --- 
The kWh usage for the current period as shown in the image is 55.


In [11]:
directory_path ='Image/water'  # Specify the path to your image
for path, folders, files  in os.walk(directory_path):
     for filename in files:
            image_path = os.path.join(directory_path, filename)
            print(image_path)
            base64_string = image_to_base64(image_path)
            Image(filename=image_path) 
            response_text = interactWithLLM(bedrock_runtime,prompt="what is the water usage in current period?",image=base64_string,llm_type=model)
            print('response_text --- \n' + response_text)


Image/water/w-Water-Bill-06-06-18.jpg
response_text --- 
According to the "Consumption History" section of the utility bill, the current water usage for the billing period with the Bill Date listed as 05-17 is 10 (units not specified, but likely gallons or cubic feet).
Image/water/w-waterbill-iowa.png
response_text --- 
According to the utility bill shown in the image, the current period water usage is 7560, which appears under the "USAGE" column for both the "WATER" and "SEWER" line items.
Image/water/w-Water Bill sample image.jpg
response_text --- 
According to the meter reading details on the water bill, the Current Read is 66 and the Previous Read is 20. The Multiplier is listed as N/A, which typically means a multiplier of 1. Therefore, the Consumption, which represents the water usage for the current billing period, is 46 units.
